Computing 2-Selmer ranks of CN curves with n upto 2,000,000

In [ ]:
import csv
from sage.all import *
from sage.parallel.decorate import parallel

# Parameters
input_file = "sq2mil.csv"
output_file = "selmer_out.csv"
batch_size = 1000
num_cores = 15  # Adjust to match your CPU

# ----------------------------
# Parallel function to compute 2-Selmer rank
# ----------------------------
@parallel(ncpus=num_cores)
def compute_selmer2(n):
    try:
        E = EllipticCurve([0, 0, 0, -n^2, 0])
        selmer_rank = E.selmer_rank()
        return (n, selmer_rank)
    except Exception as e:
        return (n, None)

# ----------------------------
# Read 'n' from CSV in batches, skip header
# ----------------------------
def read_batches_with_header(filename, batch_size):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header
        assert header[0].strip().lower() == 'n'
        batch = []
        for row in reader:
            if not row: continue
            try:
                n = int(row[0].strip())
                batch.append(n)
            except:
                continue
            if len(batch) == batch_size:
                yield batch
                batch = []
        if batch:
            yield batch

# ----------------------------
# Process batches in parallel
# ----------------------------
all_results = []

for batch in read_batches_with_header(input_file, batch_size):
    for (_, result) in compute_selmer2(batch):
        n, selmer = result
        if selmer is not None:
            all_results.append((n, selmer))

# ----------------------------
# Sort and write to output
# ----------------------------
all_results.sort(key=lambda x: x[0])

with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["n", "selmer2_rank"])
    writer.writerows(all_results)
